In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time
import os
import json

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

# Getting Pokemon and their Stats

In [3]:
page = requests.get('https://www.serebii.net/pokemon/all.shtml')

In [4]:
soup = BeautifulSoup(page.content, 'html.parser')

In [5]:
table = soup.find(class_='dextable')

In [6]:
heads = table.find_all('td', class_='fooevo')
heads = [i.text for i in heads]
heads = [re.sub('[\t|\r|\n]','',i) for i in heads]
heads.remove('Base Stats')
heads.remove('Pic')
heads

['No.',
 'Name',
 'Type',
 'Abilities',
 'HP',
 'Att',
 'Def',
 'S.Att',
 'S.Def',
 'Spd']

In [7]:
# Making Column for Generation of Pokemon
gens = {'gen1': range(1, 152),
        'gen2': range(152, 252),
        'gen3': range(252, 387),
        'gen4': range(387, 494),
        'gen5': range(494, 650),
        'gen6': range(650, 722),
        'gen7': range(722, 810),
        'gen8': range(810, 899)}
gen_list = []
for h,i in enumerate(gens.values()):
    for j in range(len(i)):
        gen_list.append(h+1)
len(gen_list)

898

In [8]:
#Getting Evolution Tiers
evo_soup = BeautifulSoup(requests.get('https://pokemon.fandom.com/wiki/List_of_Pok%C3%A9mon_by_evolution').content, 'html.parser')
evo_rows = evo_soup.find_all('tr')

In [9]:
E = [i.find_all('td') for i in evo_rows if i.find_all('td') != []]
E

[[<td>1
  </td>,
  <td><a href="/wiki/Bulbasaur" title="Bulbasaur">Bulbasaur</a>
  </td>,
  <td><a href="/wiki/Ivysaur" title="Ivysaur">Ivysaur</a>
  </td>,
  <td><a href="/wiki/Venusaur" title="Venusaur">Venusaur</a>
  </td>],
 [<td>2
  </td>,
  <td><a href="/wiki/Charmander" title="Charmander">Charmander</a>
  </td>,
  <td><a href="/wiki/Charmeleon" title="Charmeleon">Charmeleon</a>
  </td>,
  <td><a href="/wiki/Charizard" title="Charizard">Charizard</a>
  </td>],
 [<td>3
  </td>,
  <td><a href="/wiki/Squirtle" title="Squirtle">Squirtle</a>
  </td>,
  <td><a href="/wiki/Wartortle" title="Wartortle">Wartortle</a>
  </td>,
  <td><a href="/wiki/Blastoise" title="Blastoise">Blastoise</a>
  </td>],
 [<td>4
  </td>,
  <td><a href="/wiki/Caterpie" title="Caterpie">Caterpie</a>
  </td>,
  <td><a href="/wiki/Metapod" title="Metapod">Metapod</a>
  </td>,
  <td><a href="/wiki/Butterfree" title="Butterfree">Butterfree</a>
  </td>],
 [<td>5
  </td>,
  <td><a href="/wiki/Weedle" title="Weedle">Wee

In [10]:
EE = [[j.find_all('a') for j in i if j.find('a') != None] for i in E]
# [each tree] - [each mon] - [each mon as string]
basic = []
stage1 = []
stage2 = []

for f in EE:
    for i,j in enumerate(f):
        for k in j:
            targets = [basic,stage1,stage2]
            targets[i].append(k.get('title'))

In [30]:
x = table.find_all('tr')[2:]
poke = [i.text.split('\n') for i in x]
sub = '[\t|\r|\n]'
poke = [[re.sub(sub,'',i) for i in j if len(re.sub(sub,'',i))>=1 and i != ' '] for j in poke if len(j)>1]

rows = table.find_all('tr')
all_links = [i.find_all('a') for i in rows]
all_links = [i for i in all_links if len(i)>0]
all_links = ['/'.join([i.get('href') for i in a]) for a in all_links]
all_types = [re.findall('/type/(\w+)',i) for i in all_links if len(re.findall('/type/(\w+)',i)) >= 1]
all_types
names = [i[1] for i in poke]
pokemon_stats = pd.DataFrame({
    'ID':[i[0] for i in poke],
    heads[1]:names,
    'Gen':gen_list,
    'Stage':[('Basic' if i in basic else 'Stage1' if i in stage1 else 'Stage2') for i in names],
    heads[2]:all_types,
    heads[3]:[i[2] for i in poke],
    f"Base_{heads[4]}":[i[3] for i in poke],
    f"Base_{heads[5]}":[i[4] for i in poke],
    f"Base_{heads[6]}":[i[5] for i in poke],
    f"Base_{heads[7]}":[i[6] for i in poke],
    f"Base_{heads[8]}":[i[7] for i in poke],
    f"Base_{heads[9]}":[i[8] for i in poke]
})

In [31]:
pokemon_stats.head()

,ID,Name,Gen,Stage,Type,Abilities,Base_HP,Base_Att,Base_Def,Base_S.Att,Base_S.Def,Base_Spd
0,#001,Bulbasaur,1,Basic,"[grass, poison]",Overgrow Chlorophyll,45,49,49,65,65,45
1,#002,Ivysaur,1,Stage1,"[grass, poison]",Overgrow Chlorophyll,60,62,63,80,80,60
2,#003,Venusaur,1,Stage2,"[grass, poison]",Overgrow Chlorophyll,80,82,83,100,100,80
3,#004,Charmander,1,Basic,[fire],Blaze Solar Power,39,52,43,60,50,65
4,#005,Charmeleon,1,Stage1,[fire],Blaze Solar Power,58,64,58,80,65,80


In [32]:
pokemon_stats['Type'] = pokemon_stats['Type'].apply(lambda x: [i.capitalize() for i in x])
pokemon_stats['ID'] = pd.to_numeric(pokemon_stats['ID'].str.replace('#',''))

In [33]:
pokemon_stats.head()

,ID,Name,Gen,Stage,Type,Abilities,Base_HP,Base_Att,Base_Def,Base_S.Att,Base_S.Def,Base_Spd
0,1,Bulbasaur,1,Basic,"[Grass, Poison]",Overgrow Chlorophyll,45,49,49,65,65,45
1,2,Ivysaur,1,Stage1,"[Grass, Poison]",Overgrow Chlorophyll,60,62,63,80,80,60
2,3,Venusaur,1,Stage2,"[Grass, Poison]",Overgrow Chlorophyll,80,82,83,100,100,80
3,4,Charmander,1,Basic,[Fire],Blaze Solar Power,39,52,43,60,50,65
4,5,Charmeleon,1,Stage1,[Fire],Blaze Solar Power,58,64,58,80,65,80


In [34]:
pokemon_stats['Type1'] = [i[0] for i in pokemon_stats['Type']]
pokemon_stats['Type2'] = [i[1] if len(i)==2 else float('NaN') for i in pokemon_stats['Type']]
pokemon_stats = pokemon_stats.drop(columns=['Type'])
pokemon_stats.head()

,ID,Name,Gen,Stage,Abilities,Base_HP,Base_Att,Base_Def,Base_S.Att,Base_S.Def,Base_Spd,Type1,Type2
0,1,Bulbasaur,1,Basic,Overgrow Chlorophyll,45,49,49,65,65,45,Grass,Poison
1,2,Ivysaur,1,Stage1,Overgrow Chlorophyll,60,62,63,80,80,60,Grass,Poison
2,3,Venusaur,1,Stage2,Overgrow Chlorophyll,80,82,83,100,100,80,Grass,Poison
3,4,Charmander,1,Basic,Blaze Solar Power,39,52,43,60,50,65,Fire,NaN
4,5,Charmeleon,1,Stage1,Blaze Solar Power,58,64,58,80,65,80,Fire,NaN


# Getting Pokemon Abilities
Will modify the dataframe above with the data scraped here before saving both

In [35]:
ability_url = 'https://pokemondb.net/ability'
ability_page = requests.get(ability_url)
soup = BeautifulSoup(ability_page.content, 'html.parser')
rows = soup.find_all('tr')

In [36]:
headers = [i.text for i in rows[0].find_all('th')]
headers

['Name', 'Pokémon', 'Description', 'Gen.']

In [37]:
#removing extra empty list items with less code
ab_table = soup.find('table', id='abilities')
rows = ab_table.find_all('tr')
abilities = [i.text.split('\n')[1:-1] for i in rows[1:]]
abilities

[['Adaptability', '13', 'Powers up moves of the same type.', '4'],
 ['Aerilate', '2', 'Turns Normal-type moves into Flying-type moves.', '6'],
 ['Aftermath', '8', 'Damages the attacker landing the finishing hit.', '4'],
 ['Air Lock', '1', 'Eliminates the effects of weather.', '3'],
 ['Analytic', '12', 'Boosts move power when the Pokémon moves last.', '5'],
 ['Anger Point', '9', 'Maxes Attack after taking a critical hit.', '4'],
 ['Anticipation', '14', "Senses a foe's dangerous moves.", '4'],
 ['Arena Trap', '3', 'Prevents the foe from fleeing.', '3'],
 ['Aroma Veil',
  '4',
  'Protects allies from attacks that limit their move choices.',
  '6'],
 ['As One', '2', 'Combines Unnerve and Chilling Neigh/Grim Neigh', '8'],
 ['Aura Break', '2', 'Reduces power of Dark- and Fairy-type moves.', '6'],
 ['Bad Dreams', '1', "Reduces a sleeping foe's HP.", '4'],
 ['Ball Fetch', '1', 'Retrieves a Poké Ball from a failed throw.', '8'],
 ['Battery', '1', "Raises power of teammates' Special moves.", '7'

Ignoring the Count of pokemon columns

In [38]:
ability_df = pd.DataFrame({
    'Ability':[i[0] for i in abilities],
    'Description':[i[2] for i in abilities],
    'Gen_Introduced':[i[3] for i in abilities]
})
ability_df

,Ability,Description,Gen_Introduced
0,Adaptability,Powers up moves of the same type.,4
1,Aerilate,Turns Normal-type moves into Flying-type moves.,6
2,Aftermath,Damages the attacker landing the finishing hit.,4
3,Air Lock,Eliminates the effects of weather.,3
4,Analytic,Boosts move power when the Pokémon moves last.,5
...,...,...,...
261,White Smoke,Prevents other Pokémon from lowering its stats.,3
262,Wimp Out,Switches out when HP drops below half.,7
263,Wonder Guard,Only supereffective moves will hit.,3
264,Wonder Skin,Makes status-changing moves more likely to miss.,5


In [39]:
pokemon_stats.head()

,ID,Name,Gen,Stage,Abilities,Base_HP,Base_Att,Base_Def,Base_S.Att,Base_S.Def,Base_Spd,Type1,Type2
0,1,Bulbasaur,1,Basic,Overgrow Chlorophyll,45,49,49,65,65,45,Grass,Poison
1,2,Ivysaur,1,Stage1,Overgrow Chlorophyll,60,62,63,80,80,60,Grass,Poison
2,3,Venusaur,1,Stage2,Overgrow Chlorophyll,80,82,83,100,100,80,Grass,Poison
3,4,Charmander,1,Basic,Blaze Solar Power,39,52,43,60,50,65,Fire,NaN
4,5,Charmeleon,1,Stage1,Blaze Solar Power,58,64,58,80,65,80,Fire,NaN


In [40]:
from itertools import product, permutations

In [41]:
all_abilities = [i for i in ability_df['Ability']]
ability_sep = []
for i in pokemon_stats['Abilities']:
    no1 = i.split(' ')
    no2 = [' '.join(f) for f in permutations(i.split(), r=2)]
    no3 = [' '.join(f) for f in permutations(i.split(), r=3)]
    
    mon = []
    for j in no1:
        if j in all_abilities and j not in mon:
            mon.append(j)
    for j in no2:
        if j in all_abilities and j not in mon:
            mon.append(j)
    for j in no3:
        if j in all_abilities and j not in mon:
            mon.append(j)
            
    ability_sep.append(mon)

In [42]:
ability_sep

[['Overgrow', 'Chlorophyll'],
 ['Overgrow', 'Chlorophyll'],
 ['Overgrow', 'Chlorophyll'],
 ['Blaze', 'Solar Power'],
 ['Blaze', 'Solar Power'],
 ['Blaze', 'Solar Power'],
 ['Torrent', 'Rain Dish'],
 ['Torrent', 'Rain Dish'],
 ['Torrent', 'Rain Dish'],
 ['Shield Dust', 'Run Away'],
 ['Shed Skin'],
 ['Tinted Lens'],
 ['Shield Dust', 'Run Away'],
 ['Shed Skin'],
 ['Swarm', 'Sniper'],
 ['Keen Eye', 'Tangled Feet', 'Big Pecks'],
 ['Keen Eye', 'Tangled Feet', 'Big Pecks'],
 ['Keen Eye', 'Tangled Feet', 'Big Pecks'],
 ['Guts', 'Hustle', 'Run Away'],
 ['Guts', 'Hustle', 'Run Away'],
 ['Sniper', 'Keen Eye'],
 ['Sniper', 'Keen Eye'],
 ['Intimidate', 'Unnerve', 'Shed Skin'],
 ['Intimidate', 'Unnerve', 'Shed Skin'],
 ['Static', 'Lightning Rod'],
 ['Static', 'Lightning Rod'],
 ['Sand Veil', 'Sand Rush'],
 ['Sand Veil', 'Sand Rush'],
 ['Rivalry', 'Hustle', 'Poison Point'],
 ['Rivalry', 'Hustle', 'Poison Point'],
 ['Rivalry', 'Poison Point', 'Sheer Force'],
 ['Rivalry', 'Hustle', 'Poison Point'],
 ['

In [43]:
pokemon_stats = pokemon_stats.drop(columns=['Abilities'])
for i in range(max([len(i) for i in ability_sep])):
    col = f"Ability{i+1}"
    pokemon_stats[col] = [j[i] if len(j) > i else float('NaN') for j in ability_sep]

In [44]:
pokemon_stats.head()

,ID,Name,Gen,Stage,Base_HP,Base_Att,Base_Def,Base_S.Att,Base_S.Def,Base_Spd,Type1,Type2,Ability1,Ability2,Ability3,Ability4
0,1,Bulbasaur,1,Basic,45,49,49,65,65,45,Grass,Poison,Overgrow,Chlorophyll,NaN,NaN
1,2,Ivysaur,1,Stage1,60,62,63,80,80,60,Grass,Poison,Overgrow,Chlorophyll,NaN,NaN
2,3,Venusaur,1,Stage2,80,82,83,100,100,80,Grass,Poison,Overgrow,Chlorophyll,NaN,NaN
3,4,Charmander,1,Basic,39,52,43,60,50,65,Fire,NaN,Blaze,Solar Power,NaN,NaN
4,5,Charmeleon,1,Stage1,58,64,58,80,65,80,Fire,NaN,Blaze,Solar Power,NaN,NaN


In [45]:
pokemon_stats.to_csv('pokemon_data.csv', index=False)

In [46]:
ability_df.to_csv('ability_df.csv', index=False)

# Pokemon Moves
More complicated of a scrape

In [47]:
move_url = 'https://pokemondb.net/move/all'
move_page = requests.get(move_url)
soup = BeautifulSoup(move_page.content, 'html.parser')

In [48]:
move_table = soup.find('table', id='moves')
move_table

<table class="data-table block-wide" id="moves">
<thead>
<tr>
<th class="sorting" data-sort="string"><div class="sortwrap">Name</div></th> <th class="sorting" data-sort="string"><div class="sortwrap">Type</div></th> <th class="sorting" data-sort="string"><div class="sortwrap">Cat.</div></th> <th class="sorting" data-blanks="1" data-sort="int"><div class="sortwrap">Power</div></th> <th class="sorting" data-blanks="1" data-sort="int"><div class="sortwrap">Acc.</div></th> <th class="sorting" data-blanks="1" data-sort="int"><div class="sortwrap">PP</div></th> <th class="sorting" data-blanks="1" data-sort="int"><div class="sortwrap">TM</div></th> <th><div class="sortwrap">Effect</div></th> <th class="sorting" data-blanks="1" data-sort="int"><div class="sortwrap">Prob. (%)</div></th> </tr>
</thead>
<tbody>
<tr>
<td class="cell-name"><a class="ent-name" href="/move/10000000-volt-thunderbolt" title="View details for 10,000,000 Volt Thunderbolt">10,000,000 Volt Thunderbolt</a></td><td class="ce

In [49]:
move_rows = move_table.find_all('tr')
move_heads = [i.text for i in move_rows[0].find_all('th')]
move_names = [i.text for i in move_table.find_all('a',class_='ent-name')]
move_type = [i.text for i in move_table.find_all('td', class_='cell-icon') if i.text != '—' and len(i.text)>0]
move_cat = [''.join((i.get('data-sort-value'),i.text)) for i in move_table.find_all('td', class_='cell-icon text-center')]
move_numbers = [[i.text for i in x.find_all(class_='cell-num')] for x in move_table.find_all('tr')[1:]]
move_numbers = [['—' if i == '' else i for i in n] for n in move_numbers]
move_desc = [i.text for i in move_table.find_all('td',class_='cell-long-text')]
move_heads

['Name', 'Type', 'Cat.', 'Power', 'Acc.', 'PP', 'TM', 'Effect', 'Prob. (%)']

In [50]:
all_moves = pd.DataFrame({
    move_heads[0]:move_names,
    move_heads[1]:move_type,
    move_heads[2]:move_cat,
    move_heads[3]:[i[0] for i in move_numbers],
    move_heads[4]:[i[1] for i in move_numbers],
    move_heads[5]:[i[2] for i in move_numbers],
    move_heads[6]:[i[3] for i in move_numbers],
    'Prob(%)':[i[4] for i in move_numbers],
    move_heads[7]:move_desc

})
all_moves.head()

,Name,Type,Cat.,Power,Acc.,PP,TM,Prob(%),Effect
0,"10,000,000 Volt Thunderbolt",Electric,special,195,—,1,—,—,Pikachu-exclusive Z-Move. High critical hit ra...
1,Absorb,Grass,special,20,100,25,—,—,User recovers half the HP inflicted on opponent.
2,Accelerock,Rock,physical,40,100,20,—,—,User attacks first.
3,Acid,Poison,special,40,100,30,—,10,May lower opponent's Special Defense.
4,Acid Armor,Poison,status,—,—,20,—,—,Sharply raises user's Defense.


In [51]:
for i in ['Type','Power','Acc.','PP','TM','Effect','Prob(%)']:
    all_moves[i] = all_moves[i].str.replace('—', 'NaN')
all_moves.head()

,Name,Type,Cat.,Power,Acc.,PP,TM,Prob(%),Effect
0,"10,000,000 Volt Thunderbolt",Electric,special,195,NaN,1,NaN,NaN,Pikachu-exclusive Z-Move. High critical hit ra...
1,Absorb,Grass,special,20,100,25,NaN,NaN,User recovers half the HP inflicted on opponent.
2,Accelerock,Rock,physical,40,100,20,NaN,NaN,User attacks first.
3,Acid,Poison,special,40,100,30,NaN,10,May lower opponent's Special Defense.
4,Acid Armor,Poison,status,NaN,NaN,20,NaN,NaN,Sharply raises user's Defense.


In [52]:
all_moves.to_csv('all_pokemon_moves.csv', index=False)

# Who can learn what moves?

In [53]:
#test
move_soup = BeautifulSoup(requests.get(f'https://pokemondb.net/move/acid#move-level').content, 'html.parser')
categories = move_soup.find_all('h2')
mons = move_soup.find_all(class_='infocard-list infocard-list-pkmn-md')

In [54]:
ids = ['move-level', 'move-tm', 'move-tr', 'move-egg', 'move-natpower']
[i.get('id') for i in move_soup.find_all('h2') if i.get('id') in ids]

['move-level', 'move-egg']

In [56]:
def form_string(item):
    for i,k in enumerate(item.text.split()):
        if k.startswith('#'):
            return i
        elif i > 0 and k == item.text.split()[0]:
            return i

In [57]:
table = mons[0].find_all(class_='infocard')
[(i.text.split()[:form_string(i)],' '.join(i.text.split()[-2:])) for i in table]

[(['Oddish'], 'Level 4'),
 (['Gloom'], 'Level 1'),
 (['Vileplume'], 'Level 1'),
 (['Tentacool'], 'Level 4'),
 (['Tentacruel'], 'Level 1'),
 (['Slowpoke', 'Galarian'], 'Level 6'),
 (['Slowbro', 'Galarian'], 'Level 1'),
 (['Bellossom'], 'Level 1'),
 (['Slowking', 'Galarian'], 'Level 1'),
 (['Lileep'], 'Level 4'),
 (['Cradily'], 'Level 1'),
 (['Tympole'], 'Level 4'),
 (['Palpitoad'], 'Level 1'),
 (['Seismitoad'], 'Level 1'),
 (['Shelmet'], 'Level 4'),
 (['Accelgor'], 'Level 1'),
 (['Skrelp'], 'Level 5'),
 (['Dragalge'], 'Level 1'),
 (['Nihilego'], 'Level 5'),
 (['Poipole'], 'Level 1'),
 (['Naganadel'], 'Level 1'),
 (['Toxel'], 'Level 1'),
 (['Toxtricity', 'Low', 'Key', 'Form'], 'Level 1'),
 (['Toxtricity', 'Amped', 'Form'], 'Level 1')]

In [58]:
[' '.join(i.text.split()[:2]) if i.text.split()[0].endswith('.') else i.text.split()[0] for i in mons[1].find_all(class_='infocard')]

['Shuckle']

In [59]:
move_learning = []
failed = []
for j in all_moves['Name']:
    search = '-'.join(j.lower().replace(',','').split())
    one_move = {}
    one_move['move'] = j
    print(j)
    try:
        url = f'https://pokemondb.net/move/{search}#move-level'
        time.sleep(1)
        move_soup = BeautifulSoup(requests.get(url).content, 'html.parser')
        ids = ['move-level', 'move-tm', 'move-tr', 'move-egg', 'move-natpower']
        cats = [i.get('id') for i in move_soup.find_all('h2') if i.get('id') in ids]
        
        mons = move_soup.find_all(class_='infocard-list infocard-list-pkmn-md')
        
        for ind,class_id in enumerate(cats):
            table = mons[ind].find_all(class_='infocard')
            if class_id =='move-level':
                one_move[class_id] = [' '.join((' '.join(i.text.split()[:form_string(i)]),' '.join(i.text.split()[-2:]))) for i in table]
            else:
                one_move[class_id] = [' '.join(i.text.split()[:form_string(i)]) for i in table]
        move_learning.append(one_move)
    except:
        failed.append(j)


10,000,000 Volt Thunderbolt
Absorb
Accelerock
Acid
Acid Armor
Acid Downpour
Acid Spray
Acrobatics
Acupressure
Aerial Ace
Aeroblast
After You
Agility
Air Cutter
Air Slash
All-Out Pummeling
Ally Switch
Amnesia
Anchor Shot
Ancient Power
Apple Acid
Aqua Jet
Aqua Ring
Aqua Tail
Arm Thrust
Aromatherapy
Aromatic Mist
Assist
Assurance
Astonish
Astral Barrage
Attack Order
Attract
Aura Sphere
Aura Wheel
Aurora Beam
Aurora Veil
Autotomize
Avalanche
Baby-Doll Eyes
Baddy Bad
Baneful Bunker
Barrage
Barrier
Baton Pass
Beak Blast
Beat Up
Behemoth Bash
Behemoth Blade
Belch
Belly Drum
Bestow
Bide
Bind
Bite
Black Hole Eclipse
Blast Burn
Blaze Kick
Blizzard
Block
Bloom Doom
Blue Flare
Body Press
Body Slam
Bolt Beak
Bolt Strike
Bone Club
Bone Rush
Bonemerang
Boomburst
Bounce
Bouncy Bubble
Branch Poke
Brave Bird
Breaking Swipe
Breakneck Blitz
Brick Break
Brine
Brutal Swing
Bubble
Bubble Beam
Bug Bite
Bug Buzz
Bulk Up
Bulldoze
Bullet Punch
Bullet Seed
Burn Up
Burning Jealousy
Buzzy Buzz
Calm Mind
Camouflage


Steel Wing
Sticky Web
Stockpile
Stoked Sparksurfer
Stomp
Stomping Tantrum
Stone Edge
Stored Power
Storm Throw
Strange Steam
Strength
Strength Sap
String Shot
Struggle
Struggle Bug
Stuff Cheeks
Stun Spore
Submission
Substitute
Subzero Slammer
Sucker Punch
Sunny Day
Sunsteel Strike
Super Fang
Superpower
Supersonic
Supersonic Skystrike
Surf
Surging Strikes
Swagger
Swallow
Sweet Kiss
Sweet Scent
Swift
Switcheroo
Swords Dance
Synchronoise
Synthesis
Tackle
Tail Glow
Tail Slap
Tail Whip
Tailwind
Take Down
Tar Shot
Taunt
Tearful Look
Teatime
Techno Blast
Tectonic Rage
Teeter Dance
Telekinesis
Teleport
Terrain Pulse
Thief
Thousand Arrows
Thousand Waves
Thrash
Throat Chop
Thunder
Thunder Cage
Thunder Fang
Thunder Punch
Thunder Shock
Thunder Wave
Thunderbolt
Thunderous Kick
Tickle
Topsy-Turvy
Torment
Toxic
Toxic Spikes
Toxic Thread
Transform
Tri Attack
Trick
Trick Room
Trick-or-Treat
Triple Axel
Triple Kick
Trop Kick
Trump Card
Twineedle
Twinkle Tackle
Twister
U-turn
Uproar
V-create
Vacuum Wave
V

In [61]:
move_learning

[{'move': '10,000,000 Volt Thunderbolt'},
 {'move': 'Absorb',
  'move-level': ['Zubat Level 1',
   'Golbat Level 1',
   'Oddish Level 1',
   'Gloom Level 1',
   'Vileplume Level 1',
   'Exeggcute Level 1',
   'Exeggutor Level 1',
   'Exeggutor Alolan Level 1',
   'Tangela Level 1',
   'Kabuto Level 1',
   'Kabutops Level 1',
   'Crobat Level 1',
   'Bellossom Level 1',
   'Lotad Level 3',
   'Lombre Level 1',
   'Ludicolo Level 1',
   'Seedot Level 3',
   'Nuzleaf Level 1',
   'Shiftry Level 1',
   'Nincada Level 21',
   'Ninjask Level 23',
   'Shedinja Level 23',
   'Roselia Level 1',
   'Budew Level 1',
   'Roserade Level 1',
   'Tangrowth Level 1',
   'Cottonee Level 1',
   'Whimsicott Level 1',
   'Petilil Level 1',
   'Lilligant Level 1',
   'Maractus Level 1',
   'Foongus Level 1',
   'Amoonguss Level 1',
   'Frillish Level 1',
   'Jellicent Level 1',
   'Joltik Level 1',
   'Galvantula Level 1',
   'Shelmet Level 1',
   'Accelgor Level 1',
   'Goomy Level 1',
   'Sliggoo Level 1

In [62]:
failed

[]

In [63]:
with open('full_movedata_backup.js', 'w') as js:
    js.write(json.dumps(move_learning))

In [78]:
pd.DataFrame(move_learning).tail(10)

,move,move-level,move-egg,move-tm,move-tr,move-natpower
831,Work Up,"[Tauros Level 5, Riolu Level 16, Lucario Level...",NaN,NaN,"[Bulbasaur, Ivysaur, Venusaur, Charmander, Cha...",NaN
832,Worry Seed,"[Bulbasaur Level 30, Ivysaur Level 40, Venusau...","[Treecko, Seedot, Nuzleaf, Shiftry, Cottonee, ...",NaN,NaN,NaN
833,Wrap,"[Tentacool Level 8, Tentacruel Level 1, Lickit...",NaN,NaN,NaN,NaN
834,Wring Out,"[Bellsprout Level 50, Weepinbell Level 58, Ten...","[Omanyte, Omastar, Chikorita, Bayleef, Meganiu...",NaN,NaN,NaN
835,X-Scissor,"[Scyther Level 40, Pinsir Level 32, Scizor Lev...",NaN,NaN,"[Sandshrew, Sandshrew Alolan, Sandslash, Sands...",NaN
836,Yawn,"[Slowpoke Level 9, Slowpoke Galarian Level 9, ...","[Squirtle, Wartortle, Blastoise, Psyduck, Gold...",NaN,NaN,NaN
837,Zap Cannon,"[Magnemite Level 52, Magneton Level 64, Porygo...",NaN,NaN,NaN,NaN
838,Zen Headbutt,"[Psyduck Level 18, Golduck Level 18, Slowpoke ...",NaN,NaN,"[Squirtle, Wartortle, Blastoise, Clefairy, Cle...",NaN
839,Zing Zap,"[Togedemaru Level 40, Pincurchin Level 50]",NaN,NaN,NaN,NaN
840,Zippy Zap,NaN,NaN,NaN,NaN,NaN


# Getting Pokemon Character Details

In [66]:
for i in pokemon_stats['Name']:
    print(i)

Bulbasaur
Ivysaur
Venusaur
Charmander
Charmeleon
Charizard
Squirtle
Wartortle
Blastoise
Caterpie
Metapod
Butterfree
Weedle
Kakuna
Beedrill
Pidgey
Pidgeotto
Pidgeot
Rattata
Raticate
Spearow
Fearow
Ekans
Arbok
Pikachu
Raichu
Sandshrew
Sandslash
Nidoran♀
Nidorina
Nidoqueen
Nidoran♂
Nidorino
Nidoking
Clefairy
Clefable
Vulpix
Ninetales
Jigglypuff
Wigglytuff
Zubat
Golbat
Oddish
Gloom
Vileplume
Paras
Parasect
Venonat
Venomoth
Diglett
Dugtrio
Meowth
Persian
Psyduck
Golduck
Mankey
Primeape
Growlithe
Arcanine
Poliwag
Poliwhirl
Poliwrath
Abra
Kadabra
Alakazam
Machop
Machoke
Machamp
Bellsprout
Weepinbell
Victreebel
Tentacool
Tentacruel
Geodude
Graveler
Golem
Ponyta
Rapidash
Slowpoke
Slowbro
Magnemite
Magneton
Farfetch'd
Doduo
Dodrio
Seel
Dewgong
Grimer
Muk
Shellder
Cloyster
Gastly
Haunter
Gengar
Onix
Drowzee
Hypno
Krabby
Kingler
Voltorb
Electrode
Exeggcute
Exeggutor
Cubone
Marowak
Hitmonlee
Hitmonchan
Lickitung
Koffing
Weezing
Rhyhorn
Rhydon
Chansey
Tangela
Kangaskhan
Horsea
Seadra
Goldeen
Seaking

In [67]:
base_url = "https://www.serebii.net/pokedex-swsh/"

In [68]:
deet = requests.get("https://pokemondb.net/pokedex/bulbasaur").content

In [69]:
deetsoup = BeautifulSoup(deet, 'html.parser')
deetsoup.find('div',class_='tabset-moves-game sv-tabs-wrapper')

<div class="tabset-moves-game sv-tabs-wrapper">
<div class="sv-tabs-tab-list">
<a class="sv-tabs-tab active" href="#tab-moves-18">Sword/Shield</a>
<a class="sv-tabs-tab" href="#tab-moves-17">Let's Go Pikachu/Let's Go Eevee</a>
</div>
<div class="sv-tabs-panel-list">
<div class="sv-tabs-panel active" id="tab-moves-18">
<div class="grid-row"> <div class="grid-col span-lg-6"><h3>Moves learnt by level up</h3> <p class="text-small"><em>Bulbasaur</em> learns the following moves in Pokémon Sword &amp; Shield at the levels specified.</p> <div class="resp-scroll"><table class="data-table"><thead><tr><th class="sorting" data-sort="int"><div class="sortwrap">Lv.</div></th> <th class="sorting" data-sort="string"><div class="sortwrap">Move</div></th> <th class="sorting" data-sort="string"><div class="sortwrap">Type</div></th> <th class="sorting" data-sort="string"><div class="sortwrap">Cat.</div></th> <th class="sorting" data-blanks="1" data-sort="int"><div class="sortwrap">Power</div></th> <th cla

In [70]:
deetsoup.find('img').get('src')

'https://img.pokemondb.net/artwork/bulbasaur.jpg'

In [71]:
re.findall('h3>(.*)h3',str(deetsoup.find('div', class_='tabset-moves-game sv-tabs-wrapper')))

['Moves learnt by level up</',
 'Egg moves</',
 'Move Tutor moves</',
 'Moves learnt by TM</',
 'Moves learnt by TR</',
 'Moves learnt by level up</',
 'Moves learnt by TM</']

# get_moves is broken for some characters that have unique div classes
* Toxtricity
* Indeedee
* Urshifu
* Calyrex

Fail with the following function, these pokemon have different forms, could be cause for the issue

In [72]:
def get_moves(soup):
    move_cats = [i.text for i in soup.find('div', class_='tabset-moves-game sv-tabs-wrapper').find_all('h3')]
    move_tables = soup.find(class_='tabset-moves-game sv-tabs-wrapper').find_all('tbody')

    output = {}
    all_h3s = []
    for i,j in enumerate(move_tables):
        category = move_cats[i]
        if category in output.keys():
            break
        else:
            output[category] = []
            
            moves = [[f.text for f in h.find_all('td') if f.text != ''] for h in j]
            if category.endswith('level up') or category.endswith('TM') or category.endswith('TR'):
                moves = [[f for f in h[:2]] for h in moves]
            else:
                moves = [h[0] for h in moves]
            
            
            for x in moves:
                output[category].append(x)
    return output

# Getting Moves for the Failed Mons

In [73]:
rare = BeautifulSoup(requests.get("https://pokemondb.net/pokedex/toxitricity").content, 'html.parser')

In [74]:
len(rare.find(class_='tabset-moves-game sv-tabs-wrapper').find_all('h3'))

6

In [75]:
len(rare.find(class_='tabset-moves-game sv-tabs-wrapper').find_all(class_='sv-tabs-panel active'))

4

In [ ]:
def get_moves_rare(soup):
    

In [79]:
get_moves(deetsoup)

{'Moves learnt by level up': [['1', 'Growl'],
  ['1', 'Tackle'],
  ['3', 'Vine Whip'],
  ['6', 'Growth'],
  ['9', 'Leech Seed'],
  ['12', 'Razor Leaf'],
  ['15', 'Poison Powder'],
  ['15', 'Sleep Powder'],
  ['18', 'Seed Bomb'],
  ['21', 'Take Down'],
  ['24', 'Sweet Scent'],
  ['27', 'Synthesis'],
  ['30', 'Worry Seed'],
  ['33', 'Double-Edge'],
  ['36', 'Solar Beam']],
 'Egg moves': ['Curse',
  'Ingrain',
  'Nature Power',
  'Petal Dance',
  'Skull Bash',
  'Toxic'],
 'Move Tutor moves': ['Grassy Glide'],
 'Moves learnt by TM': [['10', 'Magical Leaf'],
  ['11', 'Solar Beam'],
  ['17', 'Light Screen'],
  ['19', 'Safeguard'],
  ['21', 'Rest'],
  ['24', 'Snore'],
  ['25', 'Protect'],
  ['28', 'Giga Drain'],
  ['29', 'Charm'],
  ['31', 'Attract'],
  ['34', 'Sunny Day'],
  ['39', 'Facade'],
  ['41', 'Helping Hand'],
  ['46', 'Weather Ball'],
  ['50', 'Bullet Seed'],
  ['74', 'Venoshock'],
  ['76', 'Round'],
  ['88', 'Grassy Terrain'],
  ['94', 'False Swipe']],
 'Moves learnt by TR': [['00

In [80]:
# all move categories, some are duplicates for different games, will only pull the first one
deetsoup.find('div', class_='tabset-moves-game sv-tabs-wrapper').find_all('h3')


[<h3>Moves learnt by level up</h3>,
 <h3>Egg moves</h3>,
 <h3>Move Tutor moves</h3>,
 <h3>Moves learnt by TM</h3>,
 <h3>Moves learnt by TR</h3>,
 <h3>Moves learnt by level up</h3>,
 <h3>Moves learnt by TM</h3>]

In [81]:
# all move tables
[[j.text for j in i.find_all('td') if j.text != ''] for i in deetsoup.find(class_='tabset-moves-game sv-tabs-wrapper').find_all('tbody')[0]]


[['1', 'Growl', 'Normal', '—', '100'],
 ['1', 'Tackle', 'Normal', '40', '100'],
 ['3', 'Vine Whip', 'Grass', '45', '100'],
 ['6', 'Growth', 'Normal', '—', '—'],
 ['9', 'Leech Seed', 'Grass', '—', '90'],
 ['12', 'Razor Leaf', 'Grass', '55', '95'],
 ['15', 'Poison Powder', 'Poison', '—', '75'],
 ['15', 'Sleep Powder', 'Grass', '—', '75'],
 ['18', 'Seed Bomb', 'Grass', '80', '100'],
 ['21', 'Take Down', 'Normal', '90', '85'],
 ['24', 'Sweet Scent', 'Normal', '—', '100'],
 ['27', 'Synthesis', 'Grass', '—', '—'],
 ['30', 'Worry Seed', 'Grass', '—', '100'],
 ['33', 'Double-Edge', 'Normal', '120', '100'],
 ['36', 'Solar Beam', 'Grass', '120', '100']]

In [82]:
# vitals-table returns 7 tables from the base url
#table0 = pokedex data
#table1 = training data - ev's and experience and frienship
#table2 = breeding data - egg groups
#table3 = base stats
#table4 = game flavor text
#table5 = locations per game
#table6 = names in other languages
data_tables = deetsoup.find_all('table', class_='vitals-table')
data_tables[0]

<table class="vitals-table">
<tbody>
<tr>
<th>National №</th>
<td><strong>001</strong></td>
</tr>
<tr>
<th>Type</th>
<td>
<a class="type-icon type-grass" href="/type/grass">Grass</a> <a class="type-icon type-poison" href="/type/poison">Poison</a> </td>
</tr>
<tr>
<th>Species</th>
<td>Seed Pokémon</td>
</tr>
<tr>
<th>Height</th>
<td>0.7 m (2′04″)</td>
</tr>
<tr>
<th>Weight</th>
<td>6.9 kg (15.2 lbs)</td>
</tr>
<tr>
<th>Abilities</th>
<td><span class="text-muted">1. <a href="/ability/overgrow" title="Powers up Grass-type moves in a pinch.">Overgrow</a></span><br/><small class="text-muted"><a href="/ability/chlorophyll" title="Boosts the Pokémon's Speed in sunshine.">Chlorophyll</a> (hidden ability)</small><br/></td>
</tr>
<tr>
<th>Local №</th>
<td>001 <small class="text-muted">(Red/Blue/Yellow)</small><br/>226 <small class="text-muted">(Gold/Silver/Crystal)</small><br/>001 <small class="text-muted">(FireRed/LeafGreen)</small><br/>231 <small class="text-muted">(HeartGold/SoulSilver)</smal

In [83]:
table1_head = [i.text for i in data_tables[0].find_all('th')][:-1]
table2_head = [i.text for i in data_tables[1].find_all('th')]
table3_head = [i.text for i in data_tables[2].find_all('th')]

table1_head, table2_head, table3_head

(['National №', 'Type', 'Species', 'Height', 'Weight', 'Abilities'],
 ['EV yield', 'Catch rate', 'Base Friendship', 'Base Exp.', 'Growth Rate'],
 ['Egg Groups', 'Gender', 'Egg cycles'])

In [84]:
table1_data = [i.text for i in data_tables[0].find_all('td')]
table2_data = [i.text for i in data_tables[1].find_all('td')]
table3_data = [i.text for i in data_tables[2].find_all('td')]

table1_data, table2_data, table3_data

(['001',
  '\nGrass Poison ',
  'Seed Pokémon',
  '0.7\xa0m (2′04″)',
  '6.9\xa0kg (15.2\xa0lbs)',
  '1. OvergrowChlorophyll (hidden ability)',
  "001 (Red/Blue/Yellow)226 (Gold/Silver/Crystal)001 (FireRed/LeafGreen)231 (HeartGold/SoulSilver)080 (X/Y — Central Kalos)001 (Let's Go Pikachu/Let's Go Eevee)068 (The Isle of Armor)"],
 ['\n1 Special Attack ',
  '\n45 (5.9% with PokéBall, full HP)\n',
  '\n50 (normal)\n',
  '64',
  'Medium Slow'],
 ['Grass, Monster', '87.5% male, 12.5% female', '20 (4,884–5,140 steps)\n'])

In [85]:
[i.get('href').split('/')[-1] for i in data_tables[0].find_all('a') if i.get('href').startswith('/ability/')]

['overgrow', 'chlorophyll']

In [86]:
table2_data[0].replace('\n','')

'1 Special Attack '

In [89]:
output = {'Name':[],
         'ID':[],
         'Type':[],
         'Species':[],
         'Height':[],
         'Weight':[],
         'Abilities':[],
         'EVs_Given':[],
         'Catch_Rate':[],
         'Base_Friendship':[],
         'Egg_Groups':[],
          'Gender_Spread':[],
          'Egg_Steps':[]
         }
to_change = {'Mr. Mime': 'mr-mime',
             'Mime Jr.': 'mime-jr',
             'Flabébé': 'flabebe',
             'Type: Null': 'type-null',
             'Tapu Koko': 'tapu-koko',
             'Tapu Lele': 'tapu-lele',
             'Tapu Bulu': 'tapu-bulu',
             'Tapu Fini': 'tapu-fini',
             "Sirfetch'd": 'sirfetchd',
             'Mr. Rime': 'mr-rime'}

failed_images = []
failed_pokemon = []

try:
    os.mkdir('Artwork')
except:
    pass

for i in pokemon_stats['Name']:
    j = i
    if '♂' in i:
        j = i.replace('♂','-m')
    elif '♀' in i:
        j = i.replace('♀','-f')
    elif i in to_change.keys():
        j = to_change[i]
        
    time.sleep(1)
    base_url = 'https://pokemondb.net/pokedex/'
    url = base_url + j.lower()
    print(j)
    soup = BeautifulSoup(requests.get(url).content, 'html.parser')
    data_tables = soup.find_all('table', class_='vitals-table')
    
    try:
        img_ref = soup.find('img').get('src')
        with open(f'Artwork/{j}.jpg', 'wb') as img:
            img.write(requests.get(img_ref).content)
    except:
        failed_images.append(i)
        
        
    try:
        table1_data = [i.text for i in data_tables[0].find_all('td')][:-1]
        table2_data = [i.text for i in data_tables[1].find_all('td')][:-2]
        table3_data = [i.text for i in data_tables[2].find_all('td')]

        output['Name'].append(i)
        output['ID'].append(table1_data[0])
        output['Type'].append(table1_data[1])
        output['Species'].append(table1_data[2])
        output['Height'].append(table1_data[3].replace('\xa0',''))
        output['Weight'].append(table1_data[4].replace('\xa0',''))
        output['Abilities'].append([i.get('href').split('/')[-1] for i in data_tables[0].find_all('a') if i.get('href').startswith('/ability/')])
        output['EVs_Given'].append(table2_data[0].replace('/n',''))
        output['Catch_Rate'].append(table2_data[1].replace('/n',''))
        output['Base_Friendship'].append(table2_data[2].replace('/n',''))
        output['Egg_Groups'].append(table3_data[0])
        output['Gender_Spread'].append(table3_data[1])
        output['Egg_Steps'].append(table3_data[2])
    except:
        failed_pokemon.append(i)

Bulbasaur
Ivysaur
Venusaur
Charmander
Charmeleon
Charizard
Squirtle
Wartortle
Blastoise
Caterpie
Metapod
Butterfree
Weedle
Kakuna
Beedrill
Pidgey
Pidgeotto
Pidgeot
Rattata
Raticate
Spearow
Fearow
Ekans
Arbok
Pikachu
Raichu
Sandshrew
Sandslash
Nidoran-f
Nidorina
Nidoqueen
Nidoran-m
Nidorino
Nidoking
Clefairy
Clefable
Vulpix
Ninetales
Jigglypuff
Wigglytuff
Zubat
Golbat
Oddish
Gloom
Vileplume
Paras
Parasect
Venonat
Venomoth
Diglett
Dugtrio
Meowth
Persian
Psyduck
Golduck
Mankey
Primeape
Growlithe
Arcanine
Poliwag
Poliwhirl
Poliwrath
Abra
Kadabra
Alakazam
Machop
Machoke
Machamp
Bellsprout
Weepinbell
Victreebel
Tentacool
Tentacruel
Geodude
Graveler
Golem
Ponyta
Rapidash
Slowpoke
Slowbro
Magnemite
Magneton
Farfetch'd
Doduo
Dodrio
Seel
Dewgong
Grimer
Muk
Shellder
Cloyster
Gastly
Haunter
Gengar
Onix
Drowzee
Hypno
Krabby
Kingler
Voltorb
Electrode
Exeggcute
Exeggutor
Cubone
Marowak
Hitmonlee
Hitmonchan
Lickitung
Koffing
Weezing
Rhyhorn
Rhydon
Chansey
Tangela
Kangaskhan
Horsea
Seadra
Goldeen
Seaki

In [90]:
failed_pokemon, failed_images

([], [])

In [91]:
pokemon_details = pd.DataFrame(output)
pokemon_details = pokemon_details.drop(columns=['Type','Abilities'])
pokemon_details.head()

,Name,ID,Species,Height,Weight,EVs_Given,Catch_Rate,Base_Friendship,Egg_Groups,Gender_Spread,Egg_Steps
0,Bulbasaur,001,Seed Pokémon,0.7m (2′04″),6.9kg (15.2lbs),\n1 Special Attack,"\n45 (5.9% with PokéBall, full HP)\n",\n50 (normal)\n,"Grass, Monster","87.5% male, 12.5% female","20 (4,884–5,140 steps)\n"
1,Ivysaur,002,Seed Pokémon,1.0m (3′03″),13.0kg (28.7lbs),"\n1 Special Attack, 1 Special Defense","\n45 (5.9% with PokéBall, full HP)\n",\n50 (normal)\n,"Grass, Monster","87.5% male, 12.5% female","20 (4,884–5,140 steps)\n"
2,Venusaur,003,Seed Pokémon,2.0m (6′07″),100.0kg (220.5lbs),"\n2 Special Attack, 1 Special Defense","\n45 (5.9% with PokéBall, full HP)\n",\n50 (normal)\n,"Grass, Monster","87.5% male, 12.5% female","20 (4,884–5,140 steps)\n"
3,Charmander,004,Lizard Pokémon,0.6m (2′00″),8.5kg (18.7lbs),\n1 Speed,"\n45 (5.9% with PokéBall, full HP)\n",\n50 (normal)\n,"Dragon, Monster","87.5% male, 12.5% female","20 (4,884–5,140 steps)\n"
4,Charmeleon,005,Flame Pokémon,1.1m (3′07″),19.0kg (41.9lbs),"\n1 Special Attack, 1 Speed","\n45 (5.9% with PokéBall, full HP)\n",\n50 (normal)\n,"Dragon, Monster","87.5% male, 12.5% female","20 (4,884–5,140 steps)\n"


In [92]:
for i in pokemon_details.columns:
    if i=="Abilities":
        pass
    else:
        pokemon_details[i] = pokemon_details[i].str.replace('\n','')
pokemon_details['Egg_Steps'] = pokemon_details['Egg_Steps'].str.replace('–','-').map(lambda x: re.findall('\((.+)\)',x)[0])

In [93]:
master = {'HP_EVs':[],
          'Atk_EVs':[],
          'Def_EVs':[],
          'Sp.Atk_EVs':[],
          'Sp.Def_EVs':[],
          'Spd_EVs':[]}

for x in [i.split(', ') for i in pokemon_details['EVs_Given']]:
    mon = {'hp':0,
          'attack':0,
          'defense':0,
          'specialattack':0,
          'specialdefense':0,
          'speed':0}
    
    for y in x:
        if y.endswith(' '):
            y = y[:-1]
        val = int(y[0])
        stat = ''.join(i.lower() for i in y if i.isalpha())
        mon[stat] += val
        
    master['HP_EVs'].append(mon['hp'])
    master['Atk_EVs'].append(mon['attack'])
    master['Def_EVs'].append(mon['defense'])
    master['Sp.Atk_EVs'].append(mon['specialattack'])
    master['Sp.Def_EVs'].append(mon['specialdefense'])
    master['Spd_EVs'].append(mon['speed'])
    

In [94]:
for k,v in master.items():
    pokemon_details[k] = v
pokemon_details= pokemon_details.drop(columns=['EVs_Given'])

In [95]:
pokemon_details['Egg_Steps'] = pokemon_details['Egg_Steps'].map(lambda x: re.findall('\-(.+)\s',x)[0].replace(',',''))
pokemon_details['Catch_Rate'] = pokemon_details['Catch_Rate'].apply(lambda x: x.split()[0])
pokemon_details['Base_Friendship'] = pokemon_details['Base_Friendship'].apply(lambda x: x.split(' ')[0])

In [96]:
pokemon_details['Weight'] = pokemon_details['Weight'].map(lambda x: re.findall('\((.*)lbs\)', x)[0])
pokemon_details = pokemon_details.rename(columns={'Weight':'Weight_lbs'})

In [97]:
pokemon_details['Height'] = pokemon_details['Height'].map(lambda x: re.findall('\((.*)\)', x)[0])

In [98]:
genders = [(i.split(',')[0].replace('% male',''), i.split(',')[1].replace('% female','')) if i!='Genderless' else i for i in pokemon_details['Gender_Spread']]
pokemon_details['Male%'] = [i[0] if len(i)==2 else 'Genderless' for i in genders]
pokemon_details['Female%'] = [i[1] if len(i)==2 else 'Genderless' for i in genders]
pokemon_details = pokemon_details.drop(columns=['Gender_Spread'])

In [99]:
egg_groups = [[j for j in i.split() if not j.replace(',','').isnumeric()] for i in pokemon_details['Egg_Groups']]
pokemon_details['Egg_Group1'] = [i[0].replace(',','') for i in egg_groups]
pokemon_details['Egg_Group2'] = [i[1].replace(',','') if len(i)>1 else 'None' for i in egg_groups]
pokemon_details = pokemon_details.drop(columns=['Egg_Groups'])

In [100]:
pokemon_details['ID'] = pd.to_numeric(pokemon_details['ID'])

In [101]:
pokemon_details.head()

,Name,ID,Species,Height,Weight_lbs,Catch_Rate,Base_Friendship,Egg_Steps,HP_EVs,Atk_EVs,Def_EVs,Sp.Atk_EVs,Sp.Def_EVs,Spd_EVs,Male%,Female%,Egg_Group1,Egg_Group2
0,Bulbasaur,1,Seed Pokémon,2′04″,15.2,45,50,5140,0,0,0,1,0,0,87.5,12.5,Grass,Monster
1,Ivysaur,2,Seed Pokémon,3′03″,28.7,45,50,5140,0,0,0,1,1,0,87.5,12.5,Grass,Monster
2,Venusaur,3,Seed Pokémon,6′07″,220.5,45,50,5140,0,0,0,2,1,0,87.5,12.5,Grass,Monster
3,Charmander,4,Lizard Pokémon,2′00″,18.7,45,50,5140,0,0,0,0,0,1,87.5,12.5,Dragon,Monster
4,Charmeleon,5,Flame Pokémon,3′07″,41.9,45,50,5140,0,0,0,1,0,1,87.5,12.5,Dragon,Monster


In [102]:
pokemon_details.to_csv('pokemon_details.csv', index=False)